# 介绍如何安装cfgrib库

* Author: Evan
* date: 2023-04-12

---

1. 为避免安装库冲突，新建一个虚拟环境`conda creat -n env_cfgrib python=3.8.8`，将其命名为`env_cfgrib`
2. 激活新的虚拟环境`conda activate env_cfgrib`
3. 安装基本的库`pip install numpy pandas matplotlib xarray cfgrib`，也可以使用`conda install`命令
4. 可以使用`pip list`或`conda list`查看环境下已经安装的包
5. 如果此时打开IDE选择`env_cfgrib`环境运行命令`import cfgrib`会报错找不到`eccodes`，然而在终端查看显示已有`eccodes`
6. 不仅要`pip install eccodes`，而且要`conda install -c conda-forge eccodes`
7. 再次`conda install`完成后在IDE中`import cfgrib`不会有报错

In [1]:
import xarray as xr
import cfgrib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib import rcParams
config = {
    "font.family":'Times New Roman',
    "mathtext.fontset":'stix',
    "font.serif": ['SimSun'],
}
rcParams.update(config)

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

# 读取grib文件

In [2]:
ds = xr.open_dataset('F:/FNL/fnl_20230216_00_00.grib2',
                     engine='cfgrib',
                     backend_kwargs={
                         'filter_by_keys':{
                             'typeOfLevel':'isobaricInhPa',
                            #  'typeOfLevel':'heightAboveGround',
                            #  'typeOfLevel':'surface',
                            #  'typeOfLevel':'pressureFromGroundLayer',
                             },
                         'errors':'ignore',})
ds

<xarray.Dataset>
Dimensions:        (isobaricInhPa: 33, latitude: 181, longitude: 360)
Coordinates:
    time           datetime64[ns] ...
    step           timedelta64[ns] ...
  * isobaricInhPa  (isobaricInhPa) float64 1e+03 975.0 950.0 ... 3.0 2.0 1.0
  * latitude       (latitude) float64 90.0 89.0 88.0 87.0 ... -88.0 -89.0 -90.0
  * longitude      (longitude) float64 0.0 1.0 2.0 3.0 ... 357.0 358.0 359.0
    valid_time     datetime64[ns] ...
Data variables:
    gh             (isobaricInhPa, latitude, longitude) float32 ...
    t              (isobaricInhPa, latitude, longitude) float32 ...
    r              (isobaricInhPa, latitude, longitude) float32 ...
    q              (isobaricInhPa, latitude, longitude) float32 ...
    w              (isobaricInhPa, latitude, longitude) float32 ...
    wz             (isobaricInhPa, latitude, longitude) float32 ...
    u              (isobaricInhPa, latitude, longitude) float32 ...
    v              (isobaricInhPa, latitude, longitude) float32 ...
    absv           (isobaricInhPa, latitude, longitude) float32 ...
    o3mr           (isobaricInhPa, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2023-04-13T20:31 GRIB to CDM+CF via cfgrib-0.9.1...

# 循环读取并导出为nc

In [ ]:
path = 'F:/FNL/'
dates = np.arange(16,21)
hours = np.arange(0,20,6)

dslist = []
for date in dates:
    for hour in hours:
        file = path + 'fnl_202302' + str(date).zfill(2) + '_' + str(hour).zfill(2) + '_00.grib2'
        # datetime = pd.to_datetime('2023-02-'+str(date).zfill(2)+'T'+str(hour).zfill(2))
        ds = xr.open_dataset(file,
                     engine='cfgrib',
                     backend_kwargs={
                         'filter_by_keys':{
                             'typeOfLevel':'isobaricInhPa',
                             },
                         'errors':'ignore',})
        dslist.append(ds)

dsout = xr.concat(dslist,dim='time')
